# Economicos

In [ ]:
%%capture
import sys
!ln -s ../syntheticml .
!{sys.executable} -m pip install -U -r ../requirements.txt
!{sys.executable} -m pip install -U kaleido

In [1]:
import pandas as pd
df = pd.read_parquet("/workspaces/synthetic-data-for-text/datasets/economicos/synth-b-1/data/tddpm_mlp_with_text.parquet")

In [7]:
from pylatexenc.latexencode import unicode_to_latex

for e in df.description.sample(5):
    print(e.replace("_", "\_"))

Casa 3 dormitorios, 2 baños (principal en suite) Living comedor Cocina Americana bodega Antejardín Trasero Terraza Cercana al supermercado Jumbo. A pasos de locomoción colectiva Los Cuevas con Avda Quilpué Sector tranquilo Lo Ovalle sector La Paz Cuentacon estacionamiento techado para dos autos más 1 portón pequeñisimo Te invitamos! Contactar: +56947923434 Fonó:993450547 Corredora Propiedades Mónica Alcántara-Corregidora +56 9 9629 9917 Email contacto@propiedadesalcandaracorredoreshipotecariosdelcampohotmailcvnrlazdelimavalco Estamos interesados
Departamento de 55 m2 en San Francisco, Las Condes. Consta con living comedor por separado uno amplio para ampliar un baño completo; Cocina amoblada equipada (hornos empotrados), hornito & campana incluidos 2 estacionamientos 1 bodega Valor: $35.000 mensuales Código Axtep 4642 Edificio cuenta entre las siguientes funciones principales que incluyen acceso controlado 24 horas Cercano al metro Manquehue Además se reciben visitas cercanas lavanderí

In [1]:
import pandas as pd
import numpy as np
df = pd.read_parquet('../datasets/economicos/raw/full_dedup_economicos_step0.parquet');
df.sample(3)

,url,description,price,property_type,transaction_type,state,county,publication_date,rooms,bathrooms,m_built,m_size,source,title,address,owner,_price
674341,https://www.economicos.cl/propiedades/espectac...,"Oportunidad única, se vende local de 1000 met...",146500 UF,Local o Casa comercial,Venta,Coquimbo,La Serena,2021-09-06 23:13:59,NaN,NaN,600.0,1000.0,None,ESPECTACULAR LOCAL COMERCIAL FRENTE A LA PLAYA,"Avenida del mar 2100 La Serena, Coquimbo",Verónica Propiedades spa,146500.000000
449454,https://www.economicos.cl/propiedades/departam...,"500.000 Luminosos departamento, cercano al met...",$ 500.000,Departamento,Arriendo,Metropolitana de Santiago,Las Condes,2019-06-12 00:06:00,2.0,2.0,65.0,NaN,El Mercurio,Departamento en Arriendo en Las Condes 2 dormi...,"Las Condes, Metropolitana de Santiago",,17.983457
735759,https://www.economicos.cl/propiedades/departam...,Dpto. con 3 dormitorios más estacionamiento c...,$ 380.000,Departamento,Arriendo,Metropolitana de Santiago,La Florida,2020-12-12 00:04:00,3.0,2.0,60.0,60.0,El Mercurio,Departamento en Arriendo en La Florida 3 dormi...,"La Florida, Metropolitana de Santiago",MUNOZ CASTILLO RAUL,13.063734


In [7]:
from pylatexenc.latexencode import unicode_to_latex
k = df.copy()

In [8]:
k["price"] = df["price"].apply(unicode_to_latex)
k["price"]

0             \$ 75.000
1               9595 UF
2            \$ 280.000
3         \$ 29.990.000
4             \$ 55.000
              ...      
760002       \$ 320.000
760004       \$ 350.000
760006       \$ 230.000
760007       \$ 350.000
760008     \$ 1.200.000
Name: price, Length: 682338, dtype: object

In [ ]:
%%capture
from syntheticml.data.synthetic import Synthetic, MODELS
category_columns=("property_type", "transaction_type", "state", "county", "rooms", "bathrooms", "m_built", "m_size", "source", )
# TODO: Estudiar implicancia de valores nulos en categorias y numeros
df_converted = df.fillna(dict(
        property_type = "None",
        transaction_type = "None",
        state = "None",
        county = "None",
        rooms = -1,
        bathrooms = -1,
        m_built = -1,
        m_size = -1,
        source = "None"
)).fillna(-1).astype({k: 'str' for k in ("description", "price", "title", "address", "owner",)})
df_converted = df.replace(to_replace="None", value=np.nan).replace(to_replace=-1, value=np.nan).dropna().astype({k: 'str' for k in ("description", "price", "title", "address", "owner",)})
basedate = pd.Timestamp('2017-12-01')
dtime = df_converted.pop("publication_date")
df_converted["publication_date"] = dtime.apply(lambda x: (x - basedate).days)
syn = Synthetic(df_converted, 
        id="url", 
        category_columns=category_columns,
        text_columns=("description", "price", "title", "address", "owner", ),
        exclude_columns=tuple(),
        synthetic_folder = "../datasets/economicos/synth",
        models=['copulagan', 'tvae', 'gaussiancopula', 'ctgan', 'smote-enc', 'tddpm_mlp'],
        n_sample = df_converted.shape[0],
        target_column="_price",
        max_cpu_pool=1
)


In [8]:
%%capture
syn.process()
#syn.process(remaining_columns=("property_type", "transaction_type", "state",))
syn.process_scores()


KeyboardInterrupt: 

## Metricas
### Continuas

In [ ]:
metrics = syn.current_metrics()
metrics[~metrics.is_categorical].dropna(axis=1, how='all')

### Categoricas

In [ ]:
metrics[metrics.is_categorical].dropna(axis=1, how='all')

## Scores

In [ ]:
#best_model = "smote-enc_noise_682338"
best_model = "tddpm_mlp_27574"
second_best= "smote-enc_27574"
avg = syn.scores[syn.scores["type"] == "avg"]
avg.sort_values("score", ascending=False).loc[["tddpm_mlp_27574","smote-enc_27574","copulagan_27574","ctgan_27574","gaussiancopula_noise_27574","tvae_noise_27574","tvae_27574","ctgan_noise_27574","copulagan_noise_27574","gaussiancopula_27574"],["score", "DCR ST 5th", "DCR SH 5th", "DCR TH 5th"]]

In [ ]:
from syntheticml.data.charts import Charts
for fig in syn.get_charts(best_model, {'date', 'id', 'zipcode', 'lat', 'long', 'yr_renovated'}):
    if fig:
        display(fig.show("png"))

In [ ]:
syn.charts.pair_corr(df.loc[:, syn.fake_data[best_model].columns].dropna(), syn.fake_data[best_model].astype({k:np.float64 for k in ('bathrooms','rooms','m_built','m_size')}), {'id', 'waterfront', 'yr_renovated'}, "_price").update_layout(dict(width=1000)).show("png")

In [ ]:
syn.fake_data[best_model]

In [ ]:
syn.fake_data[second_best]

In [ ]:
from syntheticml.data.charts import Charts
for fig in syn.get_charts(second_best, {'date', 'id', 'zipcode', 'lat', 'long', 'yr_renovated'}):
    if fig:
        display(fig.show("png"))